[simple autoencoder](https://blog.keras.io/building-autoencoders-in-keras.html)

In [10]:
%matplotlib inline
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
tf.__version__

'2.1.0'

In [12]:
train = pd.read_csv("S1-ADL1.dat",sep=" ",header=None)
train.fillna(0,inplace=True)

feat_columns = [c for c in train.columns if c not in [249,248,247,246,245,244,243,219,220,221,0]]
target_column = [c for c in train.columns if c in [249,248,247,246,245,244,243]]
feat_columns


[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [13]:
train[target_column]

,243,244,245,246,247,248,249
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0


### Normalise training data

In [14]:
def norm(x):
    x_max = max(x)
    x_min = min(x)
    
    return (x-x_min)/(x_max-x_min)

In [70]:
for f in feat_columns:
    train[f] = norm(train[f])
    
X_train, X_test, Y_train, Y_test = train_test_split(train[feat_columns], train[target_column], test_size=0.3)

In [71]:
X_train.fillna(0,inplace=True)
X_test.fillna(0,inplace=True)
X_train.isna().sum().sum()
X_test.shape

(15335, 239)

In [32]:
# reduce to 40 features
encoding_dim = 2

input_df = Input(shape=(239,))
encoded = Dense(encoding_dim, activation='relu')(input_df)
decoded = Dense(239, activation='sigmoid')(encoded)

# encoder
autoencoder = Model(input_df, decoded)

# intermediate result
encoder = Model(input_df, encoded)

autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

In [33]:
autoencoder.fit(X_train, X_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))

Train on 35781 samples, validate on 15335 samples
Epoch 1/50
35781/35781 [==============================] - 1s 20us/step - loss: 0.0422 - val_loss: 0.0204
Epoch 2/50
35781/35781 [==============================] - 1s 15us/step - loss: 0.0149 - val_loss: 0.0137
Epoch 3/50
35781/35781 [==============================] - 1s 15us/step - loss: 0.0136 - val_loss: 0.0136
Epoch 4/50
35781/35781 [==============================] - 1s 15us/step - loss: 0.0135 - val_loss: 0.0135
Epoch 5/50
35781/35781 [==============================] - 1s 15us/step - loss: 0.0135 - val_loss: 0.0135
Epoch 6/50
35781/35781 [==============================] - 1s 14us/step - loss: 0.0135 - val_loss: 0.0135
Epoch 7/50
35781/35781 [==============================] - 1s 14us/step - loss: 0.0135 - val_loss: 0.0135
Epoch 8/50
35781/35781 [==============================] - 1s 15us/step - loss: 0.0135 - val_loss: 0.0135
Epoch 9/50
35781/35781 [==============================] - 1s 15us/step - loss: 0.0134 - val_loss: 0.0134
Epoch

### Apply prediction on reduced dimensional features

In [82]:
encoded_X_train = encoder.predict(X_train)
encoded_X_test = encoder.predict(X_test)
encoded_X_train.shape
Y_test.shape

(15335, 7)

In [64]:
#y_pred = LR.predict(encoded_X_val)
#y_pred

In [36]:
import seaborn as sns 
from sklearn.preprocessing import MinMaxScaler  

In [37]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [38]:
y_pred=rfc.predict(X_test)

In [39]:
y_pred[10]

array([  2, 103,   0,   0,   0,   0,   0], dtype=int64)

In [1]:
#Y_test

In [84]:
# from sklearn.linear_model import LinearRegression
# mlr_model = LinearRegression()

# mlr_model.fit(X_train,Y_train)

# y_pred_mlr= mlr_model.predict(X_test)

# Model

In [42]:
rfc1 = RandomForestClassifier(n_estimators=200)
rfc1.fit(encoded_X_train,Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [43]:
encoded_y_pred = rfc1.predict(encoded_X_test)
encoded_y_pred.shape

(15335, 7)

In [44]:
(encoded_y_pred == Y_test).sum().sum()

91413

In [45]:
(y_pred == Y_test).sum().sum()

106857

In [75]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents_train = pca.fit_transform(X_train)
principalDf_train = pd.DataFrame(data = principalComponents_train
             , columns = ['principal component 1', 'principal component 2'])

principalComponents_test = pca.fit_transform(X_test)
principalDf_test = pd.DataFrame(data = principalComponents_test
             , columns = ['principal component 1', 'principal component 2'])

In [76]:
#principalDf_train
#Y_trainn = pd.DataFrame(data = Y_train
#             , columns = ['1', '2','3','4','5','6','7'])

In [78]:
rfc_pca = RandomForestClassifier(n_estimators=200)
rfc_pca.fit(principalDf_train,Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [79]:
pca_y_pred = rfc_pca.predict(principalDf_test)

In [80]:
(pca_y_pred == Y_test).sum().sum()

73098